# Clustering Crypto

In [1]:
!pip install -U altair

     |████████████████████████████████| 727 kB 10.6 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [88]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import json
from pathlib import Path
import numpy as np

import altair as alt


### Fetching Cryptocurrency Data

In [89]:
# Use the following endpoint to fetch json data

# url = "https://min-api.cryptocompare.com/data/all/coinlist"
# response = requests.get(url)
# json_response = response.json()
# print(json.dumps(json_response, indent=4))

## Data rate exceeded error messages after multiple pulls, so CSV data to be used

In [90]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [91]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path, index_col=["Unnamed: 0"])
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [92]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'

# Set the index to column that would have been dropped in code below (commented out), since it seems
# that is what was wanted, per screen shots in instructions. 
# crypto_df.drop(columns=["Unnamed: 0"], inplace=True)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [93]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.drop(crypto_df[crypto_df["IsTrading"] == False].index)
crypto_df.head()
# df.drop(df[df['c3'] == False].index, inplace=True)
crypto_df["IsTrading"].value_counts()

True    1144
Name: IsTrading, dtype: int64

In [94]:
# Keep only cryptocurrencies with a working algorithm
crypto_df.drop(crypto_df[crypto_df["Algorithm"] == ""].index, inplace=True)

# All cells should have an algorithm now, so it is assumed they are working algorithms. 

In [95]:
# Remove the "IsTrading" column
crypto_df.drop(columns=["IsTrading"], inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [96]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna(how="any",axis=0)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [97]:
# Remove rows with cryptocurrencies having no coins mined

# Using .replace to replace any blank strings with NaN, and then dropping NaN values
crypto_df["TotalCoinsMined"].replace("", np.nan, inplace=True)
crypto_df["TotalCoinsMined"].dropna(inplace=True)
crypto_df["TotalCoinsMined"].isnull().sum()

# Dropping values of 0 in TotalCoinsMined column, after dropping blank / Nan values
crypto_df.drop(crypto_df[crypto_df["TotalCoinsMined"] <= 0].index, inplace=True)

In [98]:
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [99]:
# Drop rows where there are 'N/A' text values

# All N/A values should be gone, but running a replace action just to double check.  Replacing 
# all "N/A" data with a recoognized NaN value and then using dropna() to eliminate. 
crypto_df.replace("N/A", np.nan, inplace=True)
crypto_df.dropna(inplace=True)

# Checking middle of dataframe to spot check for null values or 0 values for TotalCoinsMined
crypto_df.iloc[100:400]

crypto_df.isnull().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [100]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coins_name = pd.DataFrame(data=crypto_df["CoinName"], index=crypto_df.index)

# coins_name = crypto_df[["CoinName"]]
coins_name.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [101]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns=["CoinName"], inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [102]:
# Create dummy variables for text features

# Converting TotalCoinSupply to float data rather than object to avoid it being encoded with dummy
# variable and creating excess columns.
crypto_df["TotalCoinSupply"] = crypto_df["TotalCoinSupply"].apply(pd.to_numeric)
crypto_df.dtypes
X = pd.get_dummies(crypto_df)
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
# Standardize data
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
X_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Reducing Dimensions Using PCA

In [104]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

# Get two principal components for the data.
crypto_pca = pca.fit_transform(X_scaled)

In [105]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index
)
pca_df.head(10)

,PC 1,PC 2,PC 3
42,-0.336026,1.027666,-0.494478
404,-0.319360,1.027885,-0.494964
1337,2.300701,1.584515,-0.623396
BTC,-0.143570,-1.333708,0.167005
ETH,-0.142564,-1.999174,0.287315
LTC,-0.162124,-1.081338,0.010616
DASH,-0.404775,1.169053,-0.396791
XMR,-0.139989,-2.192037,0.408016
ETC,-0.141005,-1.999266,0.287290
ZEC,-0.148778,-2.140960,0.353660


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [106]:
inertia = []
k = list(range(1, 11))


# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=1)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

alt.Chart(df_elbow).mark_line(size=3).encode(
    alt.X("k"),
    alt.Y("inertia"),
    tooltip=["k", "inertia"]

).interactive()

alt.Chart(...)

Running K-Means with `k=5`

In [107]:
# Initialize the K-Means model
model = KMeans(n_clusters=5, random_state=1)

# Fit the model
model.fit(pca_df)

# Predict clusters
predictions = model.predict(pca_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
pca_df["Class"] = model.labels_
pca_df.head()

,PC 1,PC 2,PC 3,Class
42,-0.336026,1.027666,-0.494478,2
404,-0.319360,1.027885,-0.494964,2
1337,2.300701,1.584515,-0.623396,2
BTC,-0.143570,-1.333708,0.167005,0
ETH,-0.142564,-1.999174,0.287315,0


In [108]:
# Creating requested cluster_df dataframe by joining multiple dataframes previously defined. 
clustered_df = crypto_df.join(pca_df).join(coins_name)
clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.336026,1.027666,-0.494478,2,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.319360,1.027885,-0.494964,2,404Coin
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.300701,1.584515,-0.623396,2,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.143570,-1.333708,0.167005,0,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.142564,-1.999174,0.287315,0,Ethereum
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.162124,-1.081338,0.010616,0,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,2.200000e+07,-0.404775,1.169053,-0.396791,2,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,-0.139989,-2.192037,0.408016,0,Monero
ETC,Ethash,PoW,1.133597e+08,2.100000e+08,-0.141005,-1.999266,0.287290,0,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,2.100000e+07,-0.148778,-2.140960,0.353660,0,ZCash


### Visualizing Results

#### 3D-Scatter with Clusters

In [109]:
# Create a 3D-Scatter with the PCA data and the clusters

# Only 2D, due to switch to using Altair on SageMaker

alt.Chart(clustered_df).mark_circle(size=60).encode(
    alt.X("PC 1"),
    alt.Y("PC 2"),
    alt.Color("Class"),
    alt.OpacityValue(4),
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
).interactive()

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [110]:
# Table with tradable cryptos
display(clustered_df)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.336026,1.027666,-0.494478,2,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.319360,1.027885,-0.494964,2,404Coin
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.300701,1.584515,-0.623396,2,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.143570,-1.333708,0.167005,0,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.142564,-1.999174,0.287315,0,Ethereum
...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2.000000e+09,2.466816,0.776317,-0.167785,2,ZEPHYR
GAP,Scrypt,PoW/PoS,1.493105e+07,2.500000e+08,-0.334071,1.027560,-0.494510,2,Gapcoin
BDX,CryptoNight,PoW,9.802226e+08,1.400223e+09,0.325824,-2.312891,0.434244,0,Beldex
ZEN,Equihash,PoW,7.296538e+06,2.100000e+07,-0.148779,-2.140960,0.353660,0,Horizen


In [111]:
# Print the total number of tradable cryptocurrencies
tradable = clustered_df["CoinName"].value_counts().sum()
print(f"Total number of tradable cryptocurrencies: {tradable}")

Total number of tradable cryptocurrencies: 532


#### Scatter Plot with Tradable Cryptocurrencies

In [112]:
# Scale data to create the scatter plot
# scaler = StandardScaler()
scaler = MinMaxScaler()

clustered_df[["TotalCoinsMined", "TotalCoinSupply"]] = scaler.fit_transform(
    clustered_df[["TotalCoinsMined", "TotalCoinSupply"]])

# scaled_clustered = scaler.fit_transform(
#     clustered_df[["TotalCoinsMined", "TotalCoinSupply"]])

# scaled_clustered_df = pd.DataFrame(data=scaled_clustered_df, 
#                                    columns=["TotalCoinsMined", "TotalCoinSupply"], 
#                                             index=crypto_df.index)
# scaled_clustered_df = scaled_clustered_df.join(coins_name)
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,CoinName
42,Scrypt,PoW/PoS,0.000000,4.200000e-11,-0.336026,1.027666,-0.494478,2,42 Coin
404,Scrypt,PoW/PoS,0.001066,5.320000e-04,-0.319360,1.027885,-0.494964,2,404Coin
1337,X13,PoW/PoS,0.029576,3.141593e-01,2.300701,1.584515,-0.623396,2,EliteCoin
BTC,SHA-256,PoW,0.000018,2.100000e-05,-0.143570,-1.333708,0.167005,0,Bitcoin
ETH,Ethash,PoW,0.000109,0.000000e+00,-0.142564,-1.999174,0.287315,0,Ethereum


In [113]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"

alt.Chart(clustered_df).mark_circle(size=60).encode(
    alt.X("TotalCoinsMined"),
    alt.Y("TotalCoinSupply"),
    alt.Color("Class"),
    alt.OpacityValue(4),
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
).interactive()

alt.Chart(...)

In [114]:
scaler = StandardScaler()

clustered_df[["TotalCoinsMined", "TotalCoinSupply"]] = scaler.fit_transform(
    clustered_df[["TotalCoinsMined", "TotalCoinSupply"]])

In [115]:
alt.Chart(clustered_df).mark_circle(size=60).encode(
    alt.X("TotalCoinsMined"),
    alt.Y("TotalCoinSupply"),
    alt.Color("Class"),
    alt.OpacityValue(4),
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
).interactive()

alt.Chart(...)

In [116]:
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,CoinName
42,Scrypt,PoW/PoS,-0.117108,-0.152870,-0.336026,1.027666,-0.494478,2,42 Coin
404,Scrypt,PoW/PoS,-0.093970,-0.145009,-0.319360,1.027885,-0.494964,2,404Coin
1337,X13,PoW/PoS,0.524946,4.489424,2.300701,1.584515,-0.623396,2,EliteCoin
BTC,SHA-256,PoW,-0.116715,-0.152560,-0.143570,-1.333708,0.167005,0,Bitcoin
ETH,Ethash,PoW,-0.114747,-0.152870,-0.142564,-1.999174,0.287315,0,Ethereum
